<a href="https://colab.research.google.com/github/Hamedsamak66/MNIST/blob/main/CNN-MNIST-Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

In [2]:
# Load the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
# Normalize the pixel values to the range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [4]:

# Reshape the images for the CNN
X_train_cnn = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test_cnn = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [5]:
# Flatten the images for the models that require 1D input
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

In [6]:
# One-hot encode the target labels
y_train_onehot = to_categorical(y_train, 10)
y_test_onehot = to_categorical(y_test, 10)

In [7]:

# Define preprocessing techniques
preprocessing_techniques = {
    'StandardScaler': StandardScaler(),
    'MinMaxScaler': MinMaxScaler()
}

In [8]:
# Neural network model
def create_nn_model(input_shape):
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# CNN model
def create_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [9]:
# Other models
models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'SVC': SVC(),
    'NeuralNetwork': create_nn_model((28, 28)),
    'CNN': create_cnn_model((28, 28, 1))
}

In [15]:
# Function to evaluate each model with each preprocessing technique
def evaluate_models(preprocessing_techniques, models, X_train_flat, X_test_flat, X_train_cnn, X_test_cnn, y_train, y_test):
    results = []
    for prep_name, prep in preprocessing_techniques.items():
        for model_name, model in models.items():
            if model_name == 'NeuralNetwork':
                # Handle neural network separately
                model = create_nn_model((784,))
                X_train_prep = prep.fit_transform(X_train_flat)
                X_test_prep = prep.transform(X_test_flat)
                model.fit(X_train_prep, y_train_onehot, epochs=5, batch_size=32, verbose=0)
                y_pred = model.predict(X_test_prep)
                # Convert predictions to class labels
                y_pred_labels = np.argmax(y_pred, axis=1)
                accuracy = accuracy_score(y_test, y_pred_labels)
                results.append((prep_name, model_name, accuracy))
            elif model_name == 'CNN':
                # Handle CNN separately
                model = create_cnn_model((28, 28, 1))
                model.fit(X_train_cnn, y_train_onehot, epochs=5, batch_size=32, verbose=0)
                y_pred = model.predict(X_test_cnn)
                # Convert predictions to class labels
                y_pred_labels = np.argmax(y_pred, axis=1)
                accuracy = accuracy_score(y_test, y_pred_labels)
                results.append((prep_name, model_name, accuracy))
            else:
                pipeline = Pipeline([
                    (prep_name, prep),
                    (model_name, model)
                ])
                pipeline.fit(X_train_flat, y_train)
                y_pred = pipeline.predict(X_test_flat)
                accuracy = accuracy_score(y_test, y_pred)
                results.append((prep_name, model_name, accuracy))
    return results


In [16]:
# Evaluate the models
results = evaluate_models(preprocessing_techniques, models, X_train_flat, X_test_flat, X_train_cnn, X_test_cnn, y_train, y_test)

# Convert results to a DataFrame for better readability
results_df = pd.DataFrame(results, columns=['Preprocessing', 'Model', 'Accuracy'])
print(results_df)

313/313 [==============================] - 2s 6ms/step
    Preprocessing                   Model  Accuracy
0  StandardScaler  RandomForestClassifier    0.9694
1  StandardScaler                     SVC    0.9661
2  StandardScaler           NeuralNetwork    0.9688
3  StandardScaler                     CNN    0.9852
4    MinMaxScaler  RandomForestClassifier    0.9688
5    MinMaxScaler                     SVC    0.9792
6    MinMaxScaler           NeuralNetwork    0.9737
7    MinMaxScaler                     CNN    0.9869


In [17]:
# Identify the best preprocessing and model combination based on accuracy score
best_result = results_df.loc[results_df['Accuracy'].idxmax()]
print(f"Best Preprocessing Technique: {best_result['Preprocessing']}")
print(f"Best Model: {best_result['Model']}")
print(f"Best Accuracy: {best_result['Accuracy']}")

Best Preprocessing Technique: MinMaxScaler
Best Model: CNN
Best Accuracy: 0.9869
